In [1]:
# Import required modules
import cv2
import numpy as np
import os
import glob
import matplotlib.pyplot as plt
import matplotlib.cm as cm
# Define the dimensions of checkerboard
CHECKERBOARD = (8, 6)


# stop the iteration when specified
# accuracy, epsilon, is reached or
# specified number of iterations are completed.
criteria = (cv2.TERM_CRITERIA_EPS +
			cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

all_image_points = []
all_object_points = []


# Vector for 3D points
threedpoints = []

# Vector for 2D points
twodpoints = []


# 3D points real world coordinates
objectp3d = np.zeros((1, CHECKERBOARD[0]
					* CHECKERBOARD[1],
					3), np.float32)
objectp3d[0, :, :2] = np.mgrid[0:CHECKERBOARD[0],
							0:CHECKERBOARD[1]].T.reshape(-1, 2)
prev_img_shape = None


# Extracting path of individual image stored
# in a given directory. Since no path is
# specified, it will take current directory
# jpg files alone
#images = glob.glob('*.jpg')

# images = glob.glob('/home/fhc/ANU/2023 S1/honours/camera_calibration/saved_img/*png')
images = glob.glob('/home/fhc/ANU/Honours/camera_calibration_file/saved_img/*png')
#print("images {}".format(images))
for filename in images:
	
	image = cv2.imread(filename)
	#print(image)
	#cv2.imshow('Image',image)
	grayColor = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
	gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
	# Find the chess board corners
	# If desired number of corners are
	# found in the image then ret = true
	ret, corners = cv2.findChessboardCorners(
					grayColor, CHECKERBOARD,
					cv2.CALIB_CB_ADAPTIVE_THRESH
					+ cv2.CALIB_CB_FAST_CHECK +
					cv2.CALIB_CB_NORMALIZE_IMAGE)

	# If desired number of corners can be detected then,
	# refine the pixel coordinates and display
	# them on the images of checker board
	if ret == True:
		threedpoints.append(objectp3d)

		# Refining pixel coordinates
		# for given 2d points.
		corners2 = cv2.cornerSubPix(
			grayColor, corners, (11, 11), (-1, -1), criteria)

		twodpoints.append(corners2)

		# Draw and display the corners
		image = cv2.drawChessboardCorners(image,
										CHECKERBOARD,
										corners2, ret)

	cv2.imshow('img', image)
	cv2.waitKey(0)

cv2.destroyAllWindows()

h, w = image.shape[:2]


# Perform camera calibration by
# passing the value of above found out 3D points (threedpoints)
# and its corresponding pixel coordinates of the
# detected corners (twodpoints)
ret, matrix, distortion, r_vecs, t_vecs = cv2.calibrateCamera(
	threedpoints, twodpoints, grayColor.shape[::-1], None, None)


# Displaying required output
print(" Camera matrix: {}".format(matrix))

print("Distortion coefficient: {}".format(distortion))
#print(distortion)

print("Rotation Vectors: {}".format(r_vecs))
#print(r_vecs)

print("Translation Vectors: {}".format(t_vecs))
#print(t_vecs)								
print("######################reprojection error###############################")

# Perform camera calibration

ret, camera_matrix, dist_coeffs, rvecs, tvecs = cv2.calibrateCamera(all_object_points, all_image_points, gray.shape[::-1], None, None)

fig, ax = plt.subplots()
colors = cm.rainbow(np.linspace(0, 1, len(images)))

for i in range(len(all_object_points)):
    img_points, _ = cv2.projectPoints(all_object_points[i], rvecs[i], tvecs[i], camera_matrix, dist_coeffs)
    img_points_reshaped = img_points.reshape(-1, 2).astype(np.float32)
    errors = all_image_points[i].reshape(-1, 2).astype(np.float32) - img_points_reshaped
    ax.scatter(errors[:, 0], errors[:, 1], c=[colors[i]], marker='o', alpha=0.5, label=f"Image {i + 1}")

ax.set_title('Reprojection Errors Scatter Plot')
ax.set_xlabel('X-axis pixel error')
ax.set_ylabel('Y-axis pixel error')

# Add a legend
ax.legend()

ax.set_xlim(-4, 4)
ax.set_ylim(-4, 4)
plt.show()
								



ModuleNotFoundError: No module named 'cv2'